<a href="https://colab.research.google.com/github/adigew/Breast-Cancer-Detection/blob/main/DenseNet121_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

     |████████████████████████████████| 16.9 MB 33.2 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 147 kB 74.0 MB/s 
     |████████████████████████████████| 209 kB 74.5 MB/s 
     |████████████████████████████████| 181 kB 75.3 MB/s 
     |████████████████████████████████| 78 kB 5.3 MB/s 
     |████████████████████████████████| 140 kB 75.4 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 59 kB 4.5 MB/s 
     |████████████████████████████████| 745 kB 33.2 MB/s 


In [ ]:
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download...
  azureml      Serve models on Azure ML.
  db           Commands for managing an MLflow...
  deployments  Deploy MLflow models to custom...
  experiments  Manage experiments.
  gc           Permanently delete runs in the
               `deleted` lifecycle stage.

  models       Deploy MLflow models locally.
  pipelines    Run MLflow Pipelines and inspect...
  run          Run an MLflow project from the...
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.
  ui           Launch the MLflow tracking UI
               for...


In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment('DenseNet_ML')

ModuleNotFoundError: ignored

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from google.colab import drive
drive.mount('/content/gdrive')
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/gdrive


In [ ]:
from tensorflow import keras
import cv2
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import Flatten,Dropout,SpatialDropout2D,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
path = "/content/gdrive/My Drive/Breast_Cancer/Ultrasound/Dataset_BUSI_with_GT"
dir_list = [os.path.join(path,i) for i in os.listdir(path)]
size_dict = {}
for i,value in enumerate(dir_list):
    size_dict[os.listdir(path)[i]] = len(os.listdir(value))
size_dict 

{'normal': 266, 'malignant': 421, 'benign': 891}

In [ ]:
img_size = 128
img_channel = 3
X_b , Xm_b , y_b = np.zeros((437,img_size,img_size,img_channel)) , np.zeros((437,img_size,img_size,img_channel)) , np.full(437,'benign') 
X_n , Xm_n , y_n = np.zeros((133,img_size,img_size,img_channel)) , np.zeros((133,img_size,img_size,img_channel)) , np.full(133,'normal') 
X_m , Xm_m , y_m = np.zeros((210,img_size,img_size,img_channel)) , np.zeros((210,img_size,img_size,img_channel)) , np.full(210,'malignant')

In [ ]:
img1_path = os.path.join(os.path.join(path,'benign'),os.listdir(os.path.join(path,'benign'))[1])

pil_img = load_img(img1_path,color_mode = 'rgb',target_size=(img_size,img_size))
img = img_to_array(pil_img)
img_shape = img.shape
print(img_shape)

def img_num(filename):
    
    val = 0
    
    for i in range(len(filename)) :
        if filename[i] == '(' :
            while True :
                i += 1
                if filename[i] == ')' :
                    break
                val = (val*10) + int(filename[i])
            break
    
    return val

for tumor_path in dir_list :
    for image in os.listdir(tumor_path) :
        p = os.path.join(tumor_path, image)
        pil_img = load_img(p,color_mode = 'rgb',target_size=(img_size,img_size))         # read image as  grayscale ans resize it
        
        if image[-5] == ')' :                                   #if real image 
            
            if image[0] == 'b' :
                X_b[img_num(image)-1]+= img_to_array(pil_img)  # If image is real add it
            if image[0] == 'n' :                               # to X as benign , normal
                X_n[img_num(image)-1]+= img_to_array(pil_img)  # or malignant.
            if image[0] == 'm' :
                X_m[img_num(image)-1]+= img_to_array(pil_img)
        else :                                                 #else masked image
            
            if image[0] == 'b' :
                Xm_b[img_num(image)-1]+= img_to_array(pil_img)  # Similarly add the target
            if image[0] == 'n' :                               # mask to y.
                Xm_n[img_num(image)-1]+= img_to_array(pil_img)
            if image[0] == 'm' :
                Xm_m[img_num(image)-1]+= img_to_array(pil_img)

(128, 128, 3)


In [ ]:
X = np.concatenate((X_b, X_n, X_m), axis = 0)
Xm = np.concatenate((Xm_b, Xm_n, Xm_m), axis = 0)
y = np.concatenate((y_b, y_n, y_m), axis = 0)

print(X.shape)
print(Xm.shape)
print(y.shape)
X /= 255.0
Xm /= 255.0

print(X.max())
print(Xm.min())

(780, 128, 128, 3)
(780, 128, 128, 3)
(780,)
1.0
0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

encoder  = OneHotEncoder()
#y = y.toarray()
y=encoder.fit_transform(y.reshape(y.shape[0],1))

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.15,shuffle=True)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.1,stratify = y_train.toarray())
Xm_train,Xm_test,ym_train,ym_test = train_test_split(Xm,y,test_size = 0.15,shuffle=True,stratify=y.toarray())
Xm_train,Xm_val,ym_train,ym_val = train_test_split(Xm_train,ym_train,test_size=0.1,stratify = ym_train.toarray())

class_list = encoder.categories_
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(596, 128, 128, 3) (117, 128, 128, 3)
(596, 3) (117, 3)


In [ ]:
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=img_shape,
    pooling=None
)

X_feat_out = base_model.output
X_feat_flatten = Flatten()(X_feat_out)

X_feat_model = Model(inputs = base_model.input,outputs = X_feat_flatten)
X_feat_train = X_feat_model.predict(X_train)
X_feat_val = X_feat_model.predict(X_val)
X_feat_test = X_feat_model.predict(X_test)

Xm_feat_train = X_feat_model.predict(Xm_train)
Xm_feat_val = X_feat_model.predict(Xm_val)
Xm_feat_test = X_feat_model.predict(Xm_test)

4/4 [==============================] - 0s 30ms/step


In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.layers import LSTM,Bidirectional,Reshape
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,cohen_kappa_score

In [ ]:
df = pd.DataFrame(columns = ['classifier',"train_accuracy",'val_accuracy',"test_accuracy","f1_measure","kappa_score","recall","Precision"])

In [ ]:
def eval(classifier_name,y_train,y_train_pred,y_val,y_val_pred,y_true,y_pred):
    y_train = np.argmax(y_train,axis=1)
#     y_train_pred = np.argmax(y_train_pred,axis=1)
    y_val = np.argmax(y_val,axis=1)
#     y_val_pred = np.argmax(y_val_pred,axis=1)
    y_true = np.argmax(y_true,axis=1)
#     y_pred = np.argmax(y_pred,axis=1)
    
    train_accuracy = round(accuracy_score(y_train,y_train_pred),4)
    val_accuracy = round(accuracy_score(y_val,y_val_pred),4)
    test_accuracy = round(accuracy_score(y_true,y_pred),4)
    f1_measure = round(f1_score(y_true,y_pred,average='weighted'),4)
    kappa_score = round(cohen_kappa_score(y_true,y_pred),4)
    recall = round(recall_score(y_true,y_pred,average='weighted'),4)
    precision = round(precision_score(y_true,y_pred,average='weighted'),4)
    
    score = {"classifier":classifier_name,"train_accuracy":train_accuracy , "val_accuracy":val_accuracy,"test_accuracy":test_accuracy,"f1_measure":f1_measure,"kappa_score":kappa_score,"recall":recall,"precision":precision}
#     df = df.append(score,ignore_index=True)
    df.loc[len(df.index)] = score.values()
    for e,a in score.items():
        print(e,a)
    print("--"*20)
    

In [ ]:
def classifier_eval(classifier,classifier_name,X_train,y_train,X_val,y_val,X_test,y_test):
    
    classifier.fit(X_train,np.argmax(y_train,axis=1))
    
    y_train_pred = classifier.predict(X_train)
    y_val_pred = classifier.predict(X_val)
    y_test_pred = classifier.predict(X_test)
    
    eval(classifier_name,y_train,y_train_pred,y_val,y_val_pred,y_test,y_test_pred)
    

In [ ]:
names = ['SVM',
    'Random Forest',
    'AdaBoost',
    'KNN',
    'XGBoost',
    'Bagging',
    'ANN'
        ]

classifier = [
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    KNeighborsClassifier(),
    XGBClassifier(),
    BaggingClassifier(),
    MLPClassifier(max_iter = 400),
]

cls_list = zip(names,classifier)
clsm_list = zip(names,classifier)

In [ ]:
for n,c in cls_list:
  classifier_eval(c,n,X_feat_train,y_train.toarray(),X_feat_val,y_val.toarray(),X_feat_test,y_test.toarray())

In [ ]:
for l in base_model.layers:
    l.trainable = False

In [ ]:
lstm_model = Sequential()
lstm_model.add(base_model)
lstm_model.add(Reshape((base_model.output.shape[1]*base_model.output.shape[2],base_model.output.shape[3])))
lstm_model.add(LSTM(128, dropout=0.5,recurrent_dropout=0.5))
lstm_model.add(Dense(3,activation='softmax'))

lstm_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
bidir_model = Sequential()
bidir_model.add(base_model)
bidir_model.add(Reshape((base_model.output.shape[1]*base_model.output.shape[2],base_model.output.shape[3])))
bidir_model.add(Bidirectional(LSTM(128, dropout=0.5,recurrent_dropout=0.5)))
bidir_model.add(Dense(3,activation='softmax'))

bidir_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=3,restore_best_weights=True, verbose=1)

In [ ]:
def plot_history(history):

   plt.style.use("ggplot")
   acc = history.history["acc"]
   val_acc = history.history["val_acc"]
   loss = history.history["loss"]
   val_loss = history.history["val_loss"]
   x = range(1, len(acc) + 1)



   fig = plt.figure(figsize=(16, 10))
   plt.subplot(1, 2, 1)
   plt.plot(x, acc, "b", label="Training acc")
   plt.plot(x, val_acc, "r", label="Validation acc")
   plt.title("Training and validation accuracy")
   plt.legend()
   plt.subplot(1, 2, 2)
   plt.plot(x, loss, "b", label="Training loss")
   plt.plot(x, val_loss, "r", label="Validation loss")
   plt.title("Training and validation loss")
   plt.legend()
   return fig    

In [ ]:
history_lstm = lstm_model.fit(X_train,y_train.toarray(),epochs = 50,validation_data = (X_val,y_val.toarray()))
lstm_train_predict = np.argmax(lstm_model.predict(X_train),axis=1)
lstm_val_predict = np.argmax(lstm_model.predict(X_val),axis=1)
lstm_test_predict = np.argmax(lstm_model.predict(X_test),axis=1)
eval("LSTM",y_train,lstm_train_predict,y_val,lstm_val_predict,y_test,lstm_test_predict)

In [ ]:
fig = plot_history(history_lstm)

In [ ]:
history_bidir = bidir_model.fit(X_train,y_train.toarray(),epochs = 50,validation_data = (X_val,y_val.toarray()))
bidir_train_predict = np.argmax(bidir_model.predict(X_train),axis=1)
bidir_val_predict = np.argmax(bidir_model.predict(X_val),axis=1)
bidir_test_predict = np.argmax(bidir_model.predict(X_test),axis=1)
eval("Bi-dir",y_train,bidir_train_predict,y_val,bidir_val_predict,y_test,bidir_test_predict)

NameError: ignored

In [ ]:
fig = plot_history(history_bidir)

NameError: ignored

In [ ]:
df

,classifier,train_accuracy,val_accuracy,test_accuracy,f1_measure,kappa_score,recall,Precision
0,SVM,0.8909,0.7910,0.7521,0.7373,0.5678,0.7521,0.7854
1,Random Forest,1.0000,0.7463,0.7179,0.6873,0.4949,0.7179,0.7704
2,AdaBoost,0.8540,0.6866,0.6410,0.6218,0.3712,0.6410,0.6645
3,KNN,0.8087,0.6269,0.6496,0.6451,0.4227,0.6496,0.6458
4,XGBoost,1.0000,0.7761,0.7607,0.7497,0.5876,0.7607,0.7815
5,Bagging,0.9916,0.6418,0.7265,0.7158,0.5287,0.7265,0.7467
6,ANN,1.0000,0.7313,0.7692,0.7640,0.6110,0.7692,0.7787
7,LSTM,1.0000,0.7463,0.7949,0.7897,0.6575,0.7949,0.7970
8,Bi-dir,1.0000,0.7612,0.7692,0.7618,0.6097,0.7692,0.7755


In [ ]:
df = pd.DataFrame(columns = ['classifier',"train_accuracy",'val_accuracy',"test_accuracy","f1_measure","kappa_score","recall","Precision"])
for n,c in clsm_list:
    classifier_eval(c,n,Xm_feat_train,ym_train.toarray(),Xm_feat_val,ym_val.toarray(),Xm_feat_test,ym_test.toarray())

classifier SVM
train_accuracy 0.8758
val_accuracy 0.806
test_accuracy 0.8462
f1_measure 0.833
kappa_score 0.7219
recall 0.8462
precision 0.8604
----------------------------------------
classifier Random Forest
train_accuracy 1.0
val_accuracy 0.9104
test_accuracy 0.906
f1_measure 0.9036
kappa_score 0.8349
recall 0.906
precision 0.9059
----------------------------------------
classifier AdaBoost
train_accuracy 0.854
val_accuracy 0.806
test_accuracy 0.7692
f1_measure 0.7633
kappa_score 0.5948
recall 0.7692
precision 0.7608
----------------------------------------
classifier KNN
train_accuracy 0.9245
val_accuracy 0.791
test_accuracy 0.8462
f1_measure 0.8378
kappa_score 0.727
recall 0.8462
precision 0.8475
----------------------------------------
classifier XGBoost
train_accuracy 1.0
val_accuracy 0.8657
test_accuracy 0.8974
f1_measure 0.8942
kappa_score 0.8191
recall 0.8974
precision 0.8977
----------------------------------------
classifier Bagging
train_accuracy 0.9916
val_accuracy 0.8657

In [ ]:
history = lstm_model.fit(Xm_train,ym_train.toarray(),epochs = 20,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
lstm_train_predict = np.argmax(lstm_model.predict(Xm_train),axis=1)
lstm_val_predict = np.argmax(lstm_model.predict(Xm_val),axis=1)
lstm_test_predict = np.argmax(lstm_model.predict(Xm_test),axis=1)
eval("LSTM",ym_train,lstm_train_predict,ym_val,lstm_val_predict,ym_test,lstm_test_predict)

history = bidir_model.fit(Xm_train,ym_train.toarray(),epochs = 20,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
bidir_train_predict = np.argmax(bidir_model.predict(Xm_train),axis=1)
bidir_val_predict = np.argmax(bidir_model.predict(Xm_val),axis=1)
bidir_test_predict = np.argmax(bidir_model.predict(Xm_test),axis=1)
eval("Bi-LSTM",ym_train,bidir_train_predict,ym_val,bidir_val_predict,ym_test,bidir_test_predict)


Epoch 1/20
19/19 [==============================] - 12s 259ms/step - loss: 0.3586 - acc: 0.8574 - val_loss: 2.7863 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 4s 187ms/step - loss: 0.1670 - acc: 0.9480 - val_loss: 2.5141 - val_acc: 0.5672
Epoch 3/20
19/19 [==============================] - 4s 189ms/step - loss: 0.1027 - acc: 0.9681 - val_loss: 2.0927 - val_acc: 0.5672
Epoch 4/20
19/19 [==============================] - 3s 181ms/step - loss: 0.0902 - acc: 0.9664 - val_loss: 6.9566 - val_acc: 0.1642
Epoch 5/20
19/19 [==============================] - 3s 182ms/step - loss: 0.1094 - acc: 0.9664 - val_loss: 3.5676 - val_acc: 0.1642
Epoch 6/20
19/19 [==============================] - 4s 187ms/step - loss: 0.0677 - acc: 0.9782 - val_loss: 2.7578 - val_acc: 0.1642
Epoch 6: early stopping
classifier LSTM
train_accuracy 0.5587
val_accuracy 0.5672
test_accuracy 0.5641
f1_measure 0.4069
kappa_score 0.0
recall 0.5641
precision 0.3182
---------------------------------------

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


19/19 [==============================] - 16s 351ms/step - loss: 0.2411 - acc: 0.9111 - val_loss: 3.2587 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 5s 263ms/step - loss: 0.1084 - acc: 0.9530 - val_loss: 3.9569 - val_acc: 0.5672
Epoch 3/20
19/19 [==============================] - 5s 272ms/step - loss: 0.1458 - acc: 0.9513 - val_loss: 3.2264 - val_acc: 0.5672
Epoch 4/20
19/19 [==============================] - 5s 261ms/step - loss: 0.0886 - acc: 0.9530 - val_loss: 3.5324 - val_acc: 0.5672
Epoch 5/20
19/19 [==============================] - 5s 262ms/step - loss: 0.0368 - acc: 0.9832 - val_loss: 4.1689 - val_acc: 0.5672
Epoch 6/20
19/19 [==============================] - 5s 270ms/step - loss: 0.0844 - acc: 0.9748 - val_loss: 3.8226 - val_acc: 0.5672
Epoch 6: early stopping
classifier Bi-LSTM
train_accuracy 0.5587
val_accuracy 0.5672
test_accuracy 0.5641
f1_measure 0.4069
kappa_score 0.0
recall 0.5641
precision 0.3182
----------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df

,classifier,train_accuracy,val_accuracy,test_accuracy,f1_measure,kappa_score,recall,Precision
0,SVM,0.8758,0.8060,0.8462,0.8330,0.7219,0.8462,0.8604
1,Random Forest,1.0000,0.9104,0.9060,0.9036,0.8349,0.9060,0.9059
2,AdaBoost,0.8540,0.8060,0.7692,0.7633,0.5948,0.7692,0.7608
3,KNN,0.9245,0.7910,0.8462,0.8378,0.7270,0.8462,0.8475
4,XGBoost,1.0000,0.8657,0.8974,0.8942,0.8191,0.8974,0.8977
5,Bagging,0.9916,0.8657,0.8547,0.8491,0.7426,0.8547,0.8521
6,ANN,1.0000,0.8507,0.8889,0.8873,0.8067,0.8889,0.8873
7,LSTM,0.5587,0.5672,0.5641,0.4069,0.0000,0.5641,0.3182
8,Bi-LSTM,0.5587,0.5672,0.5641,0.4069,0.0000,0.5641,0.3182


In [ ]:
with mlflow.start_run(run_name="MLflow on Colab"):
  history = bidir_model.fit(Xm_train,ym_train.toarray(),epochs = 5,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
  bidir_train_predict = np.argmax(bidir_model.predict(Xm_train),axis=1)
  bidir_val_predict = np.argmax(bidir_model.predict(Xm_val),axis=1)
  bidir_test_predict = np.argmax(bidir_model.predict(Xm_test),axis=1)
  eval("Bi-LSTM",ym_train,bidir_train_predict,ym_val,bidir_val_predict,ym_test,bidir_test_predict) 
  mlflow.log_metric("acc", history.history['val_acc'][2] )
  mlflow.log_param("parameters", history.params)

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2EoUQToQgcBDS1euOrt4XKw64kx_5pyHcFMHNbZ6dz2UAtHdC"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)


Epoch 1/3
19/19 [==============================] - 4s 186ms/step - loss: 4.3925e-04 - acc: 1.0000 - val_loss: 0.3105 - val_acc: 0.9254
Epoch 2/3
19/19 [==============================] - 3s 173ms/step - loss: 0.0051 - acc: 0.9966 - val_loss: 0.2242 - val_acc: 0.9403
Epoch 3/3
19/19 [==============================] - 3s 166ms/step - loss: 0.0030 - acc: 0.9966 - val_loss: 0.2538 - val_acc: 0.9254
classifier Bi-LSTM
train_accuracy 1.0
val_accuracy 0.9254
test_accuracy 0.9744
f1_measure 0.9745
kappa_score 0.9562
recall 0.9744
precision 0.9748
----------------------------------------


IndexError: ignored

In [ ]:
history.history['val_acc']

[0.9402984976768494,
 0.9253731369972229,
 0.9402984976768494,
 0.9253731369972229,
 0.9253731369972229,
 0.9253731369972229]

In [ ]:
#!mlflow ui